# Categorization using averaged word vectors as document feature

In [27]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from pandas import DataFrame
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from sklearn.pipeline import Pipeline
import numpy as np
from nltk.corpus import stopwords as sw

stopwords = sw.words('german')

get a list of:
* the full corpora split into categories -> ```fulldata_path```
* a subset of each category corpus used for training -> ```train_paths```
* a subset of each category corpus used for validation -> ```validation_paths```

the categories were split into training / validation by using the ```mail/generateSets.py``` script with a 70 / 30 split between training and validation

In [28]:
category_names = ['Sonstiges', 'Aktuell', 'Lifestyle', 
          'Wirtschaft', 'Finanzen', 'Ausland', 'Lokal', 
          'Politik', 'Sport', 'Technologie', 'Kultur']

num_models = len(category_names)

# the list of full corpora
fulldata_paths = [(x, "corpus/corpus{}.txt".format(x)) for x in category_names]

# the corpora with a fixed split for training and validation
train_paths = [(x, "data/corpus{}.training.txt".format(x)) for x in category_names]
validation_paths = [(x, "data/corpus{}.validation.txt".format(x)) for x in category_names]

base_model = Word2Vec.load('../wiki/data/wiki.de.word2vec.model')

In [29]:
def load_sets(paths):
    X, y = [], []

    for name, path in paths:
        with open(path) as cur_file:
            for line in cur_file:
                tokens = [x for x in line.split() if x not in stopwords]
                X.append(tokens)
                y.append(name)
    return X, y

## word2vec Vectorizers
These vectorizers are used to transform a set of vectors to a single vector. They are used to transform a list of word embeddings to a single vector that represents the whole article.

Both variations simply build the average of all word-vectors. The TFIDF variation however uses the word frequency and inverse-document frequency to weight the word vectors.

The implementation in mostly adapted from [Text Classification With Word2Vec](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/) by Nadbor Drozd

The ```MeanEmbeddingVectorizer``` generates a document vector $  \overrightarrow { d } $ from a list of word vectors by calculating

$$ \overrightarrow { d } =\frac { \sum _{ i=0 }^{ dim(d) }{ \overrightarrow { { w }_{ d,i } }  }  }{ dim(d) } $$

where: 
* $ \overrightarrow { {w}_{d,i} } $ is the $ i $-th word of document $ d $

In [30]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = word2vec.vector_size
    
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

The ```TfidfEmbeddingVectorizer``` uses the same averaging strategy as the ```MeanEmbeddingVectorizer```, however it also weights every word vector $ \overrightarrow { {w}_{d,i} }$ with the term frequency-inverse document frequency (TF-IDF) of the word to put more weight on words appearing in fewer documents.

$$ \overrightarrow { d } =\frac { \sum _{ i=0 }^{ dim(d) }{ \overrightarrow { { w }_{ d,i } } *tfidf(\overrightarrow { { w }_{ d,i } } ) }  }{ dim(d) }  $$

In [31]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = word2vec.vector_size
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

a simple random forest classifier is used for classification of the document vectors

In [32]:
rf_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(base_model)), 
                        ("random forest", RandomForestClassifier(n_estimators=200))])
rf_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(base_model)), 
                        ("random forest", RandomForestClassifier(n_estimators=200))])

## Cross validation Score

in this section, the cross_val_score function of scikitlearn is used to validate the model. 

However, to be able to equally compare the different classification strategies, a fixed training and validation set is used in the next section.

In [33]:
X, y = load_sets(fulldata_paths)

/Users/daniel/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [34]:
score_rf = cross_val_score(rf_w2v, X, y, cv=2).mean()

In [35]:
score_rf_tfidf = cross_val_score(rf_w2v_tfidf, X, y, cv=2).mean()

In [37]:
print('Score simple: {}'.format(score_rf))
print('Score TFIDF:  {}'.format(score_rf_tfidf))

Score simple: 0.596355703114
Score TFIDF:  0.592372507043


## Training

In [38]:
# use only the tfidf model for further consideration since it performs slightly better in the cross validation
# however, it also needs twice the time to compute

# create a new instance to make sure the model isn't pre trained from the previous step
test_model =  Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(base_model)), 
                        ("extra trees", RandomForestClassifier(n_estimators=200))])

load the training data

In [39]:
train_X, train_y = load_sets(train_paths)

/Users/daniel/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


train the model

In [40]:
# fit returns self. assign it to a dummy variable to stop jupyter from printing the model
_ = test_model.fit(train_X, train_y)

In [41]:
validate_X, validate_y = load_sets(validation_paths)
predicted_y = test_model.predict(validate_X)

/Users/daniel/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


## Validation

This section performs the same validation steps that were used when validating the log-likelihood score approach fpr article classification, so the steps aren't as well documented. Thee the other document for a complete explanation

In [42]:
classification_matrix = np.zeros([num_models, num_models], dtype=int)

for target, predicted in zip(validate_y, predicted_y):
    target_index = category_names.index(target)
    predicted_index = category_names.index(predicted)
    classification_matrix[predicted_index, target_index] += 1
    
result = DataFrame(classification_matrix, category_names, category_names)
print(result)  

             Sonstiges  Aktuell  Lifestyle  Wirtschaft  Finanzen  Ausland  \
Sonstiges          505        3        142          36        11       30   
Aktuell              0        0          0           0         0        0   
Lifestyle           18        0         79           8         0        2   
Wirtschaft          39        7         50         631       129       31   
Finanzen             0        0          2           8       162        1   
Ausland              1        0          1           1         0       67   
Lokal                4        0          0           0         0        0   
Politik           1034       26        388        1163       397      900   
Sport                8        0          0           1         0        2   
Technologie         19        4          5          37         3        1   
Kultur              14        0         11           1         0        0   

             Lokal  Politik  Sport  Technologie  Kultur  
Sonstiges       2

## Accuracy

In [43]:
# the max(, 1) function surrounding sum makes sure wo don't divide by 0 if no match occurred
accuracy_matrix = [category / float(max([sum(category) ,1])) for category in classification_matrix]

result = DataFrame(accuracy_matrix, category_names, category_names)
print(result)  

             Sonstiges   Aktuell  Lifestyle  Wirtschaft  Finanzen   Ausland  \
Sonstiges     0.540107  0.003209   0.151872    0.038503  0.011765  0.032086   
Aktuell       0.000000  0.000000   0.000000    0.000000  0.000000  0.000000   
Lifestyle     0.134328  0.000000   0.589552    0.059701  0.000000  0.014925   
Wirtschaft    0.037072  0.006654   0.047529    0.599810  0.122624  0.029468   
Finanzen      0.000000  0.000000   0.011429    0.045714  0.925714  0.005714   
Ausland       0.008772  0.000000   0.008772    0.008772  0.000000  0.587719   
Lokal         0.028986  0.000000   0.000000    0.000000  0.000000  0.000000   
Politik       0.119759  0.003011   0.044939    0.134700  0.045981  0.104239   
Sport         0.018182  0.000000   0.000000    0.002273  0.000000  0.004545   
Technologie   0.056886  0.011976   0.014970    0.110778  0.008982  0.002994   
Kultur        0.162791  0.000000   0.127907    0.011628  0.000000  0.000000   

                Lokal   Politik     Sport  Technolo

In [44]:
true_positives = 0.0
num_samples = 0
for x in range(num_models):
    true_positives += classification_matrix[x][x]
    num_samples += sum(classification_matrix[x])
    
average_score = true_positives / num_samples

print('score: {}'.format(average_score))

score: 0.46478990201


In [45]:
# make suer we calculate the "same" accuracy as scipy
# just to prevent dumb mistakes...
score = accuracy_score(validate_y, predicted_y, normalize=True)
print('score: {}'.format(score))

score: 0.46478990201


In [46]:
# phew